In [6]:
import os

# Byt till din PythonML-mapp
os.chdir("/Users/gabrieledholm/Documents/PythonML")
print("Nuvarande arbetsmapp:", os.getcwd())

# Kontrollera att din dataset-mapp finns
dataset_path = "NewLogoPictures"
print("Finns NewLogoPictures?:", os.path.exists(dataset_path))
print("Innehåll:", os.listdir(dataset_path))

Nuvarande arbetsmapp: /Users/gabrieledholm/Documents/PythonML
Finns NewLogoPictures?: True
Innehåll: ['classes.txt', '.DS_Store', 'images', 'labels', 'notes.json']


In [7]:
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")

print("Antal bilder:", len(os.listdir(images_path)) if os.path.exists(images_path) else 0)
print("Antal label-filer:", len(os.listdir(labels_path)) if os.path.exists(labels_path) else 0)


Antal bilder: 45
Antal label-filer: 45


In [ ]:
data_yaml = f"""
train: {dataset_path}/images
val: {dataset_path}/images
nc: 1
names: ['hp_logo']
"""


yaml_path = "data.yaml"   
with open(yaml_path, "w") as f:
    f.write(data_yaml)

print("data.yaml skapad på:", yaml_path)


data.yaml skapad på: data.yaml


In [9]:
from ultralytics import YOLO

# Ladda en förtränad YOLO-modell
model = YOLO("yolov8n.pt")

# Träna modellen med din dataset
model.train(
    data="/Users/gabrieledholm/Documents/PythonML/data.yaml",  # rätt plats!
    epochs=50,
    imgsz=640,
    batch=16,
    name="newHP_Logo_train",
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.3.197 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.190 🚀 Python-3.13.5 torch-2.8.0 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/gabrieledholm/Documents/PythonML/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=newHP_Logo_train, nbs=64, nms=False, opset=None, opt

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x132b46c10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [ ]:
import os
import cv2
from ultralytics import YOLO

# ----------------------
# Inställningar
# ----------------------
video_input = "/Users/gabrieledholm/Documents/PythonML/Pitstop.mp4"  # Originalvideo
video_output = "/Users/gabrieledholm/Documents/PythonML/Pitstop_result_final1.mp4"  # Sparas här

# Säkerställ att output-mappen finns
output_dir = os.path.dirname(video_output)
os.makedirs(output_dir, exist_ok=True)

# Ladda den tränade YOLO-modellen
model = YOLO("runs/detect/newHP_Logo_train/weights/best.pt")

# Öppna videon
cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    raise RuntimeError(f"Kunde inte öppna videon: {video_input}")

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30  # default om FPS inte kan hämtas
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Skapa VideoWriter med H.264 codec
fourcc = cv2.VideoWriter_fourcc(*'avc1')
out = cv2.VideoWriter(video_output, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError(f"Kunde inte skapa output-video: {video_output}")

print("Startar bearbetning... detta kan ta lite tid beroende på videons längd.")

# ----------------------
# Processa varje frame
# ----------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO-prediction
    results = model.predict(frame, conf=0.5, verbose=False)
    annotated_frame = results[0].plot()  # Rita ut rutor på framen

    # Skriv till output-video
    out.write(annotated_frame)

# ----------------------
# Stäng filer
# ----------------------
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Färdig! Videon sparad som: {video_output}")


Startar bearbetning... detta kan ta lite tid beroende på videons längd.
Färdig! Videon sparad som: /Users/gabrieledholm/Documents/PythonML/Pitstop_result_final1.mp4


2025-09-09 16:29:20.290 python[14744:614436] WARNING: -finishWriting should not be called on the main thread.


In [5]:
import cv2
from ultralytics import YOLO
import os

video_input = "/Users/gabrieledholm/Documents/PythonML/Pitstop.mp4"
video_output = "/Users/gabrieledholm/Desktop/Pitstop_test.mp4"  # skrivbar plats

cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    raise RuntimeError("Kunde inte öppna input-video")

fps = cap.get(cv2.CAP_PROP_FPS) or 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# H.264 codec
fourcc = cv2.VideoWriter_fourcc(*'avc1')
out = cv2.VideoWriter(video_output, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError("Kunde inte skapa output-video – kolla codec och skrivbehörighet")

model = YOLO("runs/detect/HP_Logo_train/weights/best.pt")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame, conf=0.3, imgsz=640, verbose=False)
    annotated_frame = results[0].plot()
    out.write(annotated_frame)

cap.release()
out.release()
print("Färdig! Videon sparad som:", video_output)


Färdig! Videon sparad som: /Users/gabrieledholm/Desktop/Pitstop_test.mp4


2025-09-09 14:33:35.520 python[14744:614436] WARNING: -finishWriting should not be called on the main thread.


In [ ]:
import cv2
from ultralytics import YOLO
import os

video_input = "/Users/gabrieledholm/Documents/PythonML/Pitstop.mp4"
video_output = "/Users/gabrieledholm/Desktop/Pitstop_test_with_time1.mp4"

cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    raise RuntimeError("Kunde inte öppna input-video")

fps = cap.get(cv2.CAP_PROP_FPS) or 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264
out = cv2.VideoWriter(video_output, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError("Kunde inte skapa output-video – kolla codec och skrivbehörighet")

model = YOLO("runs/detect/newHP_Logo_train/weights/best.pt")

# ----------------------
# Timer variabler
# ----------------------
exposure_time = 0.0   # ackumulerad tid i sekunder
frame_time = 1.0 / fps

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, conf=0.3, imgsz=640, verbose=False)

    # Kolla om vi har någon detektion
    if len(results[0].boxes) > 0:
        exposure_time += frame_time  # lägg till tid för denna frame

    # Rita ut YOLO-resultaten
    annotated_frame = results[0].plot()

    # Lägg till timertext på bilden
    timer_text = f"Exposure time: {exposure_time:.2f} s"
    cv2.putText(
        annotated_frame,
        timer_text,
        (50, 50),  # position (x, y)
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,        # textstorlek
        (0, 0, 255),  # färg (röd)
        3           # tjocklek
    )

    out.write(annotated_frame)

cap.release()
out.release()
print("Färdig! Videon sparad som:", video_output)


2025-09-15 09:29:33.947 python[77620:1443542] AVF: AVAssetWriter status: Cannot Save


RuntimeError: Kunde inte skapa output-video – kolla codec och skrivbehörighet